# OMERO image annotation Workflow with CellPose

## 1. Setup and Installation

In [ ]:
# Import the package
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
)

# System imports
import os
from pathlib import Path

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
# Check dependencies
try:
    import ezomero
    print(f"🔗 OMERO functionality: ✅ Available")
except ImportError:
    print(f"🔗 OMERO functionality: ❌ Install with: pip install -e .[omero]")

try:
    import keyring
    print(f"🔑 Keyring support: ✅ Available")
except ImportError:
    print(f"🔑 Keyring support: ⚠️ Not available (manual password entry only)")

📦 omero-annotate-ai version: 0.1.0
🔗 OMERO functionality: ✅ Available
🔑 Keyring support: ✅ Available


## 2. OMERO Connection

In [1]:
# Create and display the OMERO connection widget
print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server:")
print("  • Fill in server details (host, username, password)")
print("  • Test connection before proceeding")
print("  • Choose password storage duration if desired")
print("  • Click 'Save & Connect' to establish connection")
print()

conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Next Step: Run the cell below after connecting to OMERO")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server:
  • Fill in server details (host, username, password)
  • Test connection before proceeding
  • Choose password storage duration if desired
  • Click 'Save & Connect' to establish connection



NameError: name 'create_omero_connection_widget' is not defined

In [3]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection established!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
print(f"🔐 Secure: {conn.isSecure()}")
print(f"🔗 Connection ready for workflow setup")

✅ OMERO connection established!
👤 User: root
🏢 Group: system
🔐 Secure: True
🔗 Connection ready for workflow setup


## 3. Workflow Configuration

In [ ]:
# Create and display the workflow widget
print("🔬 OMERO Annotation Workflow Setup")
print("Follow the sequential workflow below:")
print("  1. Select the local working directory")
print("  2. Choose OMERO container")
print("  3. Check existing annotation tables")
print("  4. Configure micro-SAM parameters")
print("  5. Save configuration")

workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

print("\n📝 Next Step: Complete the workflow above, then run the cell below")

🔬 OMERO Annotation Workflow Setup
Follow the sequential workflow below:
  1. Select working directory
  2. Choose OMERO container
  3. Check existing annotation tables
  4. Configure micro-SAM parameters
  5. Save configuration




📝 Next Step: Complete the workflow above, then run the cell below


## 4. Pipeline Execution

#### 4.1. Configuration summary

In [5]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

# Display configuration summary
print("\n📋 Configuration Summary:")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🎯 Training Set: {config.name}")
print(f"   🔬 Model: {config.ai_model.model_type}")
print(f"   📺 Channel: {config.spatial_coverage.channels}")
print(f"   📁 Output: {config.output.output_directory}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"   🧩 Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})"
    )

if config.spatial_coverage.three_d:
    print(f"   🧊 3D processing: Enabled")


print(f"\n📊 Processing: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")


📋 Configuration Summary:
   📦 Container: plate (ID: 2)
   🎯 Training Set: micro_sam_incucyte_20250909_212809
   🔬 Model: vit_b_lm
   📺 Channel: [0]
   📁 Output: D:\Maarten\Data\cellpose_tmp
   🔄 Resume from Table: False
   📖 Read-only Mode: False
   🧩 Patches: 1 per image (512×512)

📊 Processing: 3 training + 3 validation


#### 4.2. Convert configuration to an annotation pipeline

In [7]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"🔍 Validating {container_type} with ID {container_id}...")

# Validate container exists
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")

print(f"✅ Found {container_type}: {container.getName()}")
if container.getDescription():
    print(f"📝 Description: {container.getDescription()}")

🔍 Validating plate with ID 2...
✅ Found plate: incucyte


#### 4.3. Run the annotation pipeline on the files
For CellPose it will export the images for training into the local folder.

In [8]:
# Run the pipeline for CellPose
print("🚀 Starting annotation pipeline...")
try:
    table_id, processed_images = pipeline.run_cp_workflow()

    print(f"\n Images successfully loaded!")        
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

🚀 Starting annotation pipeline...
Loading images from plate 2
Found 88 images
Processing 88 images for Cellpose training
Preparing processing units...
Creating initial OMERO table...
📋 Created/replaced tracking table 'micro_sam_training_micro_sam_incucyte_20250909_212809' with 6 units
   Container: plate 2
   Table ID: 42
Created OMERO table with ID: 42
Found 6 unprocessed annotations
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 images
Loaded 2D data shape: (1040, 1408)
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 images
Loaded 2D data shape: (1040, 1408)
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 images
Loaded 2D data shape: (1040, 1408)
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1
✅ Successfully loaded 1 im

#### 4.4. Run the CellPose GUI to train on the images

In [ ]:
#Run cellpose and finetune the model
# pixi run cellpose

#### 4.5. Collect the annotations and upload to OMERO to connect to the original data

# Export as bioimage.io model
Check the details here:
https://cellpose.readthedocs.io/en/latest/models.html#sharing-models-on-bioimage-io

## 6. Cleanup

Close the OMERO connection when finished.

In [ ]:
# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print(f"\n🎉 Streamlined workflow completed!")
print(f"📊 Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")